# Using the ``fancytypes.PreSerializableAndUpdatable`` class #

## A NOTE BEFORE STARTING ##

Since the ``fancytypes`` git repository tracks this notebook under its original
basename ``using_PreSerializableAndUpdatable_class.ipynb``, we recommend that 
you copy the original notebook and rename it to any other basename that is not 
one of the original basenames that appear in the ``<root>/examples`` directory 
before executing any of the notebook cells below, where ``<root>`` is the root 
of the ``fancytypes`` repository. This way you can explore the notebook by 
executing and modifying cells without changing the original notebook, which is 
being tracked by git.

## Import necessary modules ##

In [ ]:
# For general array handling and constructing random number generators.
import numpy as np



# The library that is the subject of this demonstration.
import fancytypes

## Introduction ##

In this notebook, we use the ``fancytypes.PreSerializableAndUpdatable`` class to
define a class of "slice shufflers", which we define as objects that can
shuffle/re-order the elements in a slice of a given array. This is a somewhat
contrived example use of the ``fancytypes.PreSerializableAndUpdatable`` class,
however it is simple and complete. 

You can find the documentation for the
``fancytypes.PreSerializableAndUpdatable`` class
[here](https://mrfitzpa.github.io/fancytypes/_build/html/_autosummary/fancytypes.PreSerializableAndUpdatable.html).
It is recommended that you consult the documentation of this class as you 
explore the notebook.

## Defining the ``SliceShuffler`` class ##

We define the ``SliceShuffler`` class as a subclass of the
``fancytypes.PreSerializableAndUpdatable`` class.

In [ ]:
# Define the validation and conversion functions.
def _check_and_convert_slice_obj(ctor_params):
    slice_obj = ctor_params["slice_obj"]
    if not isinstance(slice_obj, slice):
        err_msg = ("The object ``slice_obj`` must be of type `slice`.")
        raise TypeError(err_msg)

    return slice_obj  # No conversion for this parameter/attribute.

def _check_and_convert_seed(ctor_params):
    seed = ctor_params["seed"]
    
    if seed is not None:
        try:
            seed = float(seed)
            if seed.is_integer():
                seed = int(seed)
            else:
                raise TypeError
        except:
            err_msg = ("The object ``seed`` must be an integer or `NoneType`.")
            raise TypeError(err_msg)
            
    return seed  # Conversion is implied under some circumstances.



# Define the pre-serialization functions.
def _pre_serialize_slice_obj(slice_obj):
    serializable_rep = {"start": slice_obj.start, 
                        "stop": slice_obj.stop, 
                        "step": slice_obj.step}
    
    return serializable_rep

def _pre_serialize_seed(seed):
    serializable_rep = seed
    
    return serializable_rep




# Define the de-pre-serialization functions.
def _de_pre_serialize_slice_obj(serializable_rep):
    slice_obj = slice(serializable_rep["start"], 
                      serializable_rep["stop"], 
                      serializable_rep["step"])
    
    return slice_obj

def _de_pre_serialize_seed(serializable_rep):
    seed = serializable_rep
    
    return seed



# Define the ``SliceShuffler`` class.
class SliceShuffler(fancytypes.PreSerializableAndUpdatable):
    _validation_and_conversion_funcs = \
        {"slice_obj": _check_and_convert_slice_obj, 
         "seed": _check_and_convert_seed}
    
    _pre_serialization_funcs = \
        {"slice_obj": _pre_serialize_slice_obj, 
         "seed": _pre_serialize_seed}
    
    _de_pre_serialization_funcs = \
        {"slice_obj": _de_pre_serialize_slice_obj, 
         "seed": _de_pre_serialize_seed}
    
    def __init__(self, slice_obj, seed):
        ctor_params = {"slice_obj": slice_obj, "seed": seed}
        fancytypes.PreSerializableAndUpdatable.__init__(self, ctor_params)

        # ``core_attrs`` are the "core attributes" of the instance.
        seed = self.core_attrs["seed"]
        self._random_generator = np.random.default_rng(seed)

        return None



    def shuffle(self, array):
        try:
            array = np.array(array)
        except:
            err_msg = ("The object ``array`` must be array-like.")
            raise TypeError(err_msg)
            
        slice_obj = self.core_attrs["slice_obj"]
        array_slice = array[slice_obj]
        self._random_generator.shuffle(array_slice)
        array[slice_obj] = array_slice

        return array

## Using the ``SliceShuffler`` class ##

Construct a valid instance of the ``SliceShuffler`` class and print the core attributes.

In [ ]:
slice_shuffler = SliceShuffler(slice_obj=slice(None, 6, 1), seed=5.0)

print(slice_shuffler.core_attrs)

Shuffle an array using the class.

In [ ]:
array = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
shuffled_array = slice_shuffler.shuffle(array)
shuffled_array

Update slice shuffler and print the core attributes.

In [ ]:
core_attr_subset = {"slice_obj": slice(3, None, 1)}
slice_shuffler.update(core_attr_subset)

print(slice_shuffler.core_attrs)

Shuffle the original array again.

In [ ]:
shuffled_array = slice_shuffler.shuffle(array)
shuffled_array

Update slice shuffler and print the core attributes.

In [ ]:
core_attr_subset = {"slice_obj": slice(3, 7, 1), "seed": 1.0}
slice_shuffler.update(core_attr_subset)

print(slice_shuffler.core_attrs)

Shuffle the original array again.

In [ ]:
shuffled_array = slice_shuffler.shuffle(array)
shuffled_array

Pre-serialize instance. You could then serialize the serializable representation
using the ``json``.

In [ ]:
serializable_rep = slice_shuffler.pre_serialize()
serializable_rep

Alternatively, you could serialize the instance of the class by using the
``dumps`` method.

In [ ]:
serialized_rep = slice_shuffler.dumps()
serialized_rep

You can also serialize the instance and save the result to a JSON file in one go
using the ``dump`` method.

In [ ]:
filename = "slice_shuffler.json"
slice_shuffler.dump(filename, overwrite=True)

Trying to save a serialized representation to a pre-existing that with
``overwrite==False`` will raise an exception.

In [ ]:
filename = "slice_shuffler.json"
slice_shuffler.dump(filename, overwrite=False)

Reconstruct the instance of ``SliceShuffler`` from the serialized
representation.

In [ ]:
slice_shuffler = SliceShuffler.loads(serialized_rep)
print(slice_shuffler.core_attrs)

Reconstruct the instance of ``SliceShuffler`` from the serialized
representation saved in the JSON file.

In [ ]:
slice_shuffler = SliceShuffler.load(filename)
print(slice_shuffler.core_attrs)

De-pre-serialize, i.e. construct an instance from a serializable representation.

In [ ]:
slice_shuffler = SliceShuffler.de_pre_serialize(serializable_rep)

print(slice_shuffler.core_attrs)

Try constructing instances of the ``SliceShuffler`` class with invalid
construction parameters.

In [ ]:
slice_shuffler = SliceShuffler(slice_obj=3, seed=5.0)

In [ ]:
slice_shuffler = SliceShuffler(slice_obj=slice(None, 6, 1), seed="foo")

Note that being a direct subclass of the
``fancytypes.PreSerializableAndUpdatable`` class, the ``SliceShuffler`` class
enforces validation upon construction, and supports pre-serialization and
de-pre-serialization, and updatable core attributes.

The ``fancytypes.Checkable`` class only supports validation upon constructing
instances.

The ``fancytypes.Updatable`` class supports updatable core attributes, and
validation upon constructing or updating instances, but it does not support
pre-serialization or de-pre-serialization.

The ``fancytypes.PreSerializable`` class supports pre-serialization,
de-serialization, and enforces validation upon constructing or updating
instances, but it does not support updatable core attributes.

## Testing the exception-raising features of the ``fancytypes.PreSerializableAndUpdatable`` class ##

There are no exception-raising features supported by the
``fancytypes.PreSerializableAndUpdatable`` class that have not already been
tested for the ``fancytypes.Checkable``, ``fancytypes.Updatable``, and
``fancytypes.PreSerializable`` classes in the notebooks
``using_Checkable_class.ipynb``, ``using_Updatable_class.ipynb``, and
``using_PreSerializable_class.ipynb`` respectively, found in the same directory
containing the current notebook.